In [4]:
import numpy as np
import sklearn
from sklearn import svm, grid_search
from sklearn.model_selection import train_test_split
import math 
from sklearn.model_selection import cross_val_score, GridSearchCV

In [6]:
data = []
with open('../training_data.txt') as inputfile:
    for line in inputfile:
        data.append(line.strip().split(' '))
        
words = data[0]
X_train = data[1:]
y_train = []

for i in range(len(X_train)):
    y_train.append(X_train[i][0])
    X_train[i] = X_train[i][1:]

X_train = np.array(X_train).astype(np.int)
y_train = np.array(y_train).astype(np.int)

# use cross validation to predict how good it is on the actual training set 
X_train, X_testtraining, y_train, y_testtraining = train_test_split(X_train, y_train, test_size = 0.3, random_state=1)

test = []
with open('../test_data.txt') as inputfile:
    for line in inputfile:
        test.append(line.strip().split(' '))

words_train = test[0]
X_test = test[1:]
X_test = np.array(X_test).astype(np.int)

In [11]:
clf = svm.SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))

0.903714285714


In [8]:
predictions = clf.predict(X_test)

In [10]:
with open("svm2.txt", "w") as f:
    f.write("Id,Prediction\n")
    for i in range(len(predictions)):
        f.write(str(i+1) + ',' + str(predictions[i]) + '\n')

In [7]:
def svc_param_selection(X, y, nfolds):
    gammas = [0.001, 0.01, 0.1, 1]
    Cs = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
print(svc_param_selection(X_train, y_train, 3))

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['prediction', 'recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, predictions
    print(classification_report(y_true, y_pred))
    print()